In [ ]:
# Loading the scan data

import itk
import numpy as np

scan_path = "folder"
mhd_data = itk.imread(scan_path)
# convert to numpy array
scan = np.array(itk.GetArrayFromImage(mhd_data), dtype=np.float32)
scan.clip(-1000, 1000, scan)  # clip to reasonable HU values

